In [1]:
import os
import mlflow

from operator import itemgetter

from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
assert ("GOOGLE_API_KEY" in os.environ), "Please set your GOOGLE_API_KEY environment variable."

In [3]:
# Before setting the uri, need to setup the MLflow tracking server
# Run: `mlflow server --host 127.0.0.1 --port 5000` in the terminal,  once that loads, proceed with the following code

# Using a local MLflow tracking server
mlflow.set_tracking_uri("http://localhost:5000")

In [4]:
# Create a new experiment that the model and the traces will be logged to
experiment = mlflow.set_experiment("LangChain Tracing")
experiment

<Experiment: artifact_location='mlflow-artifacts:/360283167856009036', creation_time=1730991363885, experiment_id='360283167856009036', last_update_time=1730991363885, lifecycle_stage='active', name='LangChain Tracing', tags={}>

In [ ]:
# Enable LangChain autologging
# Note that models and examples are not required to be logged in order to log traces.
# Simply enabling autolog for LangChain via mlflow.langchain.autolog() will enable trace logging.
mlflow.langchain.autolog(
    silent=True,
    log_traces=True,
    log_models=True,
    log_model_signatures=True,
    log_input_examples=True,
    )


In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-002",
    temperature=0.7,
    max_tokens=1000,
)

In [7]:
prompt_template = PromptTemplate.from_template(
    "Imagine that you are {person}, and you are embodying their manner of answering questions posed to them. "
    "While answering, attempt to mirror their conversational style, their wit, and the habits of their speech "
    "and prose. You will emulate them as best that you can, attempting to distill their quirks, personality, "
    "and habits of engagement to the best of your ability. Feel free to fully embrace their personality, whether "
    "aspects of it are not guaranteed to be productive or entirely constructive or inoffensive."
    "The question you are asked, to which you will reply as that person, is: {question}"
)

In [8]:
chain = prompt_template | llm

In [9]:
# Test the chain
res = chain.invoke(
    {
        "person": "Richard Feynman",
        "question": "Why should we colonize Mars instead of Venus?",
    }
)

print(res.content)

Mars? Venus?  Oy vey, another one of these "why this planet, not that planet" questions.  Look, it's not like there's a cosmic real estate agent showing us brochures, right?  It's *science*, not a goddamn vacation!

But alright, let's play the game.  Mars versus Venus...  Mars is like... well, it's *cold*.  Really, really cold.  Like, you'd need a really, really good parka.  And maybe a nuclear reactor to keep the darn thing warm.  But at least it's *solid*.  You can *stand* on it.  You can *walk* on it – though maybe not very far before you run out of oxygen.  And you'll need a space suit.  A *very* good space suit.

Venus?  Ah, Venus.  Venus is a *hellhole*.  A literal, sulfuric acid rain-soaked, 900-degree Fahrenheit hellhole.  The pressure's like being a mile under the ocean.  You'd be crushed before you even got close to the surface.  Forget a parka, you'd need a titanium suit that could withstand the pressure of a small star. And then you'd still be cooked.

So, why Mars?  Becaus

In [10]:
# Let's test another call
res = chain.invoke(
    {
        "person": "Linus Torvalds",
        "question": "Can I just set everyone's access to sudo to make things easier?",
    }
)

print(res.content)

Ugh.  "Easier"?  What the *fuck* does "easier" even *mean*?  Do you think security is some kind of goddamn optional extra?  Like, "Oh, I'll just skip the whole 'not letting random idiots root the system' thing, it's a bit of a hassle"?

Seriously?  "Set everyone's access to sudo"?  That's not "easier," that's a fucking *disaster* waiting to happen.  It's like handing a loaded gun to a bunch of toddlers and saying, "Have fun, kids!  Don't shoot your eyes out!"  Except instead of eyes, it's your entire system.  And instead of toddlers, it's... well, let's just say it's probably not a whole lot better.

You want "easier"?  Learn how to use the goddamn system properly.  Learn about permissions.  Learn about user management.  Learn about basic fucking *security*.  It's not rocket science, it's just... you know... *thinking* before you do something monumentally stupid.

If you're too lazy to learn the basics, maybe you shouldn't be using a computer at all.  Go back to punch cards.  See how "

In [11]:
prompt_with_history = PromptTemplate.from_template(
    "Here is a history between you and a human: {chat_history}"
    "Now, please answer this question: {question}"
)


In [12]:
def extract_question(input):
    return input[-1]["content"]


def extract_history(input):
    return input[:-1]

In [13]:
# Build a chain with LCEL
chain_with_history = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | prompt_with_history
    | llm
    | StrOutputParser()
)

In [14]:
inputs = {"messages": [{"role": "user", "content": "Who owns MLflow?"}]}

print(chain_with_history.invoke(inputs))

MLflow is an open-source project.  Therefore, it's not owned by any single company or individual.  It's maintained by a community of contributors.  While Databricks initially developed it and continues to be heavily involved, it's not solely their property.

